# Analyze Fluorescence Intensity in Sections

Performing this analysis for TCF/Lef::H2B-d2EGFP signal in m3::Dyn1(K44A)-electroporated crest

## Import Modules

In [39]:
# Import packages
import datetime
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats

# Import plotting packages
import iqplot
import bokeh.io
import matplotlib as mpl
import seaborn as sns

print("matplotlib v{}".format(mpl.__version__))
print("seaborn v{}".format(sns.__version__))
bokeh.io.output_notebook()


# Add experiment date here to apply to dataframe
now = datetime.datetime.now()
analysis_date = now.strftime("%Y%m%d")

matplotlib v3.3.2
seaborn v0.11.0


Loading BokehJS ...

## Collect Data from Fiji Analysis

In [40]:
# Navigate to CSV path
path = os.path.abspath('')+'/raw_data_csvs/'
full_df = pd.DataFrame()
list_ = []

for file_ in glob.glob(path + "/*.csv"):         # For loop to bring in files and concatenate them into a single dataframe
    df = pd.read_csv(file_)
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]                      # Determine Image name from file name
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'],                   # Split values in Image name column
         df['Somites'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    df['EmbID'] = df['ExptDate'] + '_' + df['Embryo']
    df['SectionID'] = df['EmbID'] + '_' + df['Section']
    df['Target'] = df['Label'].map(lambda x: x.split(':')[0])                       # Split values in ROI label
    df['ROI'] = df['Label'].map(lambda x: x.split(':')[1])                          # Split values in ROI label
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,IntDen,RawIntDen,Image,ExptDate,Treatment,Stains,Embryo,Somites,Section,EmbID,SectionID,Target,ROI
0,1,TCF/Lef:background:c:4/5 - 20210909_m3Dyn1KA_B...,10.976,6.798,74.614,1448.0,20210909_m3Dyn1KA_BF;Pax7;RFP;TCFLef;DAPI_Emb6...,20210909,m3Dyn1KA,BF;Pax7;RFP;TCFLef;DAPI,Emb6,8ss,sec3,20210909_Emb6,20210909_Emb6_sec3,TCF/Lef,background
1,2,TCF/Lef:background:c:4/5 - 20210909_m3Dyn1KA_B...,13.449,7.031,94.556,1835.0,20210909_m3Dyn1KA_BF;Pax7;RFP;TCFLef;DAPI_Emb6...,20210909,m3Dyn1KA,BF;Pax7;RFP;TCFLef;DAPI,Emb6,8ss,sec3,20210909_Emb6,20210909_Emb6_sec3,TCF/Lef,background
2,3,TCF/Lef:background:c:4/5 - 20210909_m3Dyn1KA_B...,23.806,7.240,172.365,3345.0,20210909_m3Dyn1KA_BF;Pax7;RFP;TCFLef;DAPI_Emb6...,20210909,m3Dyn1KA,BF;Pax7;RFP;TCFLef;DAPI,Emb6,8ss,sec3,20210909_Emb6,20210909_Emb6_sec3,TCF/Lef,background
3,4,TCF/Lef:Expt:c:4/5 - 20210909_m3Dyn1KA_BF;Pax7...,5117.603,129.235,661371.881,12834945.0,20210909_m3Dyn1KA_BF;Pax7;RFP;TCFLef;DAPI_Emb6...,20210909,m3Dyn1KA,BF;Pax7;RFP;TCFLef;DAPI,Emb6,8ss,sec3,20210909_Emb6,20210909_Emb6_sec3,TCF/Lef,Expt
4,5,TCF/Lef:Cntl:c:4/5 - 20210909_m3Dyn1KA_BF;Pax7...,6071.714,290.363,1763002.333,34213789.0,20210909_m3Dyn1KA_BF;Pax7;RFP;TCFLef;DAPI_Emb6...,20210909,m3Dyn1KA,BF;Pax7;RFP;TCFLef;DAPI,Emb6,8ss,sec3,20210909_Emb6,20210909_Emb6_sec3,TCF/Lef,Cntl


## Analyze Data
Calculates CTCF (CTCF = ROI IntDen - (background mean * ROI area)), then normalizes CTCF to the mean of the control group for each treatment in the dataframe

In [41]:
# Define control and experimental constructs
cntl_construct = 'pBS'
expt_construct = 'm3Dyn1KA'

# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean background values and group by Treatment, Embryo, Fluor, ROI and Section
mean_sections = ((full_df.groupby(['Treatment', 'Target', 'EmbID', 'Somites', 'SectionID', 'ROI'])
                  ['Area', 'Mean', 'IntDen']).mean())
mean_sections.head()

Area  \
Treatment Target  EmbID         Somites SectionID          ROI                       
m3Dyn1KA  TCF/Lef 20200807_Emb1 8ss     20200807_Emb1_sec1 Cntl        2569.175000   
                                                           Expt        2371.162000   
                                                           background     7.525333   
                                        20200807_Emb1_sec2 Cntl        1778.156000   
                                                           Expt        1581.521000   

                                                                           Mean  \
Treatment Target  EmbID         Somites SectionID          ROI                    
m3Dyn1KA  TCF/Lef 20200807_Emb1 8ss     20200807_Emb1_sec1 Cntl        2925.196   
                                                           Expt        2536.338   
                                                           background   511.915   
                                        20200807_Emb1_sec2 Cntl        3291.851   
                                                           Expt        2299.651   

                                                                             IntDen  
Treatment Target  EmbID         Somites SectionID          ROI                       
m3Dyn1KA  TCF/Lef 20200807_Emb1 8ss     20200807_Emb1_sec1 Cntl        7.515341e+06  
                                                           Expt        6.014067e+06  
                                                           background  3.890643e+03  
                                        20200807_Emb1_sec2 Cntl        5.853425e+06  
                                                           Expt        3.636948e+06

In [42]:
# Initialize for final dataframe collection
full_results = pd.DataFrame()
list_ = []

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))
    
    # Calculate CTCF
    # Calculate background (background mean * ROI area)
    background_corr_cntl = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Cntl', level='ROI')['Area'])
    background_corr_expt = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Expt', level='ROI')['Area'])
    # Slice out only Cntl or Expt values in IntDen
    intdens_cntl = df_treatment.xs('Cntl', level='ROI')['IntDen'] 
    intdens_expt = df_treatment.xs('Expt', level='ROI')['IntDen'] 
    
    # Subtract background from IntDens to determine CTCF and concatenate into single dataframe
    sub_cntl = pd.DataFrame(intdens_cntl - background_corr_cntl).reset_index()
    sub_cntl.rename(columns={0:'Cntl CTCF'}, inplace=True)
    sub_expt = pd.DataFrame(intdens_expt - background_corr_expt).reset_index()
    sub_expt.rename(columns={0:'Expt CTCF'}, inplace=True)
    full_ctcf = sub_cntl.merge(sub_expt)
    full_ctcf['CTCF Ratio'] = full_ctcf['Expt CTCF'] / full_ctcf['Cntl CTCF']
    list_.append(full_ctcf)
    
# full_results.head()
full_results_by_section = pd.concat(list_, sort=False)
# Optional: Save out data
full_results_by_section.to_csv('m3Dyn1KA_TCFLef_CTCF_Results_by_section.csv')
full_results_by_section.head()

,Target,EmbID,Somites,SectionID,Cntl CTCF,Expt CTCF,CTCF Ratio
0,TCF/Lef,20200807_Emb1,8ss,20200807_Emb1_sec1,6.200142e+06,4.800234e+06,0.774214
1,TCF/Lef,20200807_Emb1,8ss,20200807_Emb1_sec2,4.850855e+06,2.745246e+06,0.565930
2,TCF/Lef,20200807_Emb1,8ss,20200807_Emb1_sec3,3.318874e+06,3.056243e+06,0.920868
3,TCF/Lef,20200817_Emb1,8ss,20200817_Emb1_sec1,7.930794e+06,4.885535e+06,0.616021
4,TCF/Lef,20200817_Emb1,8ss,20200817_Emb1_sec2,6.335538e+06,4.176907e+06,0.659282


In [44]:
# Mean sections for a given embryo
full_results_by_bro = ((full_results_by_section.groupby(['Target', 'EmbID', 'Somites'])
                  ['Cntl CTCF', 'Expt CTCF', 'CTCF Ratio']).mean()).reset_index()

# Normalize results to the mean of the control dataset
# Note: Experiment on 20210909 imaged on a different microscope and thus should be normalized internally
normalized_list = []
sept9_experiment = full_results_by_bro.loc[full_results_by_bro['EmbID'].isin(['20210909_Emb3','20210909_Emb4','20210909_Emb5','20210909_Emb6'])]
sept9_experiment['Norm Cntl CTCF'] = sept9_experiment['Cntl CTCF']/(sept9_experiment['Cntl CTCF'].mean())
sept9_experiment['Norm Expt CTCF'] = sept9_experiment['Expt CTCF']/(sept9_experiment['Cntl CTCF'].mean())
normalized_list.append(sept9_experiment)
other_experiments = full_results_by_bro.loc[full_results_by_bro['EmbID'].isin(['20200807_Emb1','20200817_Emb1','20200821_Emb1'])]
other_experiments['Norm Cntl CTCF'] = other_experiments['Cntl CTCF']/(other_experiments['Cntl CTCF'].mean())
other_experiments['Norm Expt CTCF'] = other_experiments['Expt CTCF']/(other_experiments['Cntl CTCF'].mean())
normalized_list.append(other_experiments)

# Compile into final DF
full_results = pd.concat(normalized_list)
full_results.to_csv('m3Dyn1KA_TCFLef_CTCF_Results_by_embryo.csv')
full_results.head()

/Users/michaelpiacentino/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/michaelpiacentino/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/michaelpiacentino/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,Target,EmbID,Somites,Cntl CTCF,Expt CTCF,CTCF Ratio,Norm Cntl CTCF,Norm Expt CTCF
3,TCF/Lef,20210909_Emb3,8ss,1.324174e+06,5.957878e+05,0.486446,1.374951,0.618634
4,TCF/Lef,20210909_Emb4,8ss,6.121770e+05,4.348809e+05,0.724473,0.635652,0.451557
5,TCF/Lef,20210909_Emb5,8ss,8.124660e+05,5.741228e+05,0.703251,0.843621,0.596138
6,TCF/Lef,20210909_Emb6,8ss,1.103464e+06,5.013297e+05,0.552731,1.145777,0.520554
0,TCF/Lef,20200807_Emb1,8ss,4.789957e+06,3.533907e+06,0.753670,0.881222,0.650143
